# Token Classification Task on IPU using BERT - Fine-tuning

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) models for a token classification task. Token classification is the task of predicting a label for each token.

![Widget inference representing the NER task](images/token_classification.png)

The most common token classification tasks are:

- NER (named-entity recognition): Classify the entities in the text (for example person, organization, location).
- POS (part-of-speech tagging): Grammatically classify the tokens (for example noun, verb, adjective).
- Chunk (chunking): Grammatically classify the tokens and group them into "chunks" that go together.

We will see how to easily load a dataset for these kinds of tasks and use the `IPUTrainer` API to fine-tune a model on it.

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
| Natural language processing | Token classification | Multiple | conll2003 | Fine-tuning | 4 | 8min |

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you.

[![Run on Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://ipu.dev/3YCsqT1)

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to enable the Poplar SDK. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

## Dependencies and configuration

In order to improve usability and support for future users, Graphcore would like to collect information about the
applications and code being run in this notebook. The following information will be anonymised before being sent to Graphcore:

- User progression through the notebook
- Notebook details: number of cells, code being run and the output of the cells
- Environment details

You can disable logging at any time by running `%unload_ext graphcore_cloud_tools.notebook_logging.gc_logger` from any cell.

Install the dependencies for this notebook.

In [ ]:
%pip install "optimum-graphcore==0.7" seqeval
%pip install graphcore-cloud-tools[logger]@git+https://github.com/graphcore/graphcore-cloud-tools
%load_ext graphcore_cloud_tools.notebook_logging.gc_logger

This notebook is built to run on any token classification task, with any model checkpoint from the [🤗 Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer. You can look at the [supported frameworks](https://huggingface.co/docs/transformers/index#supported-frameworks) to confirm that the model has the necessary features. The model must also be supported by Optimum Graphcore. 

The notebook might need some small adjustments to the names of the columns used you choose to use a different dataset than the one used here. 

`max_seq_length` is the length we are going to pad the sentences to, so it should not be larger than the maximum length of the model. The IPU config files of the supported models are available in Graphcore's [🤗 account](https://huggingface.co/Graphcore). You can also create your own IPU config file locally. 

In this notebook, we are using both data parallelism and pipeline parallelism. Refer to the [tutorial on efficient data loading](https://github.com/graphcore/examples/tree/master/tutorials/pytorch/efficient_data_loading) for more information. Therefore the global batch size, which is the actual number of samples used for the weight update, is calculated from three factors:
- global batch size = micro_batch_size * gradient accumulation steps * replication factor

The replication factor is determined by the type of IPU Pod, which will be used as a key to select the replication factor from a dictionary defined in the IPU config file. For example, the dictionary in the IPU config file [Graphcore/bert-base-ipu](https://huggingface.co/Graphcore/bert-base-ipu/blob/main/ipu_config.json) looks like this:
- "replication_factor": {"pod4": 1, "pod8": 2, "pod16": 4, "pod32": 8, "pod64": 16, "default": 1}

Depending on your model and the IPU Pod you are using, you might need to adjust these three batch-size-related arguments.

Set these seven parameters, and the rest of the notebook should run smoothly:

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-uncased"
batch_size = 16

max_seq_length = 128
ipu_config_name = "Graphcore/bert-base-ipu"
micro_batch_size = 1
gradient_accumulation_steps = 16

Values for machine size and cache directories can be configured through environment variables or directly in the notebook:

In [ ]:
import os

n_ipu = int(os.getenv("NUM_AVAILABLE_IPU", 4))
executable_cache_dir = os.getenv("POPLAR_EXECUTABLE_CACHE_DIR", "/tmp/exe_cache/") + "/token_classification"

### Sharing your model with the community

You can share your model with the 🤗 community. You do this by completing the following steps:

1. Store your authentication token from the 🤗 website. [Sign up to 🤗](https://huggingface.co/join) if you haven't already.
2. Execute the following cell and input your username and password.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS to manage large files:

In [ ]:
!apt install git-lfs

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we will use for evaluation (to compare our model to the benchmark). This can be easily done with the `load_dataset` and `load_metric` functions.  

In [ ]:
from datasets import load_dataset, load_metric

For this notebook, we'll use the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf). The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or a CSV file, then you may have to make some changes to the names of the columns used. Refer to the 🤗 Datasets documentation on [loading datasets from local files](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

In [ ]:
datasets = load_dataset("conll2003")

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test sets.

In [ ]:
datasets

We can see the training, validation and test sets all have a column for the tokens (the input text split into words) and one column of labels for each kind of task we listed in the introduction.

To access an actual element, you need to select a split ("train" in the example), then specify an index:

In [ ]:
datasets["train"][0]

The labels are already coded as integer IDs to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [ ]:
datasets["train"].features[f"ner_tags"]

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' and so on. In addition to 'O' (which means no special entity), there are four labels for NER. Each label is prefixed with 'B-' (for beginning) or 'I-' (for intermediate) that indicate if the token is the first one for the current group with the label or not:
- 'PER' for person
- 'ORG' for organization
- 'LOC' for location
- 'MISC' for miscellaneous

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the `datasets` object:

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

We want to get a sense of what the data looks like, so we define the `show_random_elements` function to display some examples picked randomly from the dataset (automatically decoding the labels in passing).

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

## Preprocessing the data

Before we can feed these text samples to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will tokenize the inputs (including converting the tokens to their corresponding IDs in the pre-trained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- We get a tokenizer that corresponds to the model architecture we want to use,
- We download the vocabulary used when pre-training this specific checkpoint.

This vocabulary will be cached, so it won't be downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

The following assertion ensures that our tokenizer is a fast tokenizer (backed by Rust) from the 🤗 Tokenizers library. Fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't in the [Supported frameworks table](https://huggingface.co/docs/transformers/index#supported-frameworks).

You can directly call this tokenizer on one sentence:

In [ ]:
tokenizer("Hello, this is one sentence!")

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. The details don't matter much for what we're doing here, but they are required by the model we will instantiate later. You can learn more about them in the [tutorial on preprocessing](https://huggingface.co/transformers/preprocessing.html).

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenizer with the argument `is_split_into_words=True`:

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

Note that transformers are often pre-trained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of this:

In [ ]:
example = datasets["train"][4]
print(example["tokens"])

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

Here the words "Zwingmann" and "sheepmeat" have been split into three subtokens.

This means that we need to do some processing on our labels as the input IDs returned by the tokenizer are longer than the lists of labels our dataset contains. This is necessary because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and also because of any possible splits of words into multiple tokens:

In [ ]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [ ]:
print(tokenized_input.word_ids())

As we can see, it returns a list with the same number of elements as our processed input IDs, mapping special tokens to `None` and all other tokens to their respective word. This means we can align the labels with the processed input IDs.

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

There are two strategies to use, selected with the `label_all_tokens` parameter. Firstly, we can set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Then we can set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We use the first strategy here. If you want to use the second strategy, set `label_all_tokens` to False.

In [ ]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `padding="max_length"` (to ensure that an input shorter than the maximum length will be padded to the maximum length), `truncation=True` (to truncate text samples that are longer than the maximum length), `max_length=max_seq_length` (to set the maximum length of a sequence) and `is_split_into_words=True` (as seen above). Note that it is necessary that all the sentences have the same length since currently Graphcore's PyTorch implementation only runs in static mode. Then we align the labels with the token IDs using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        padding="max_length",
        truncation=True,
        max_length=max_seq_length,
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word ID that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several samples. In the case of several samples, the tokenizer will return a list of lists for each key:

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of the `dataset` object we created earlier. This will apply the function to all elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed with a single command.

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is able to detect when the function you pass to `map` has changed (and thus to not use the cached data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files. You can also pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the text samples together into batches. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the text samples in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pre-trained model and fine-tune it. Since we are focussing on token classification tasks, we use the `AutoModelForTokenClassification` class. As with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [ ]:
from transformers import AutoModelForTokenClassification
from optimum.graphcore import IPUConfig, IPUTrainer, IPUTrainingArguments

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

The warning tells us that we are throwing away some weights and randomly initializing others. This is normal in this case, because we are removing the head used to pre-train the model on a masked language modelling objective and replacing it with a new head for which we don't have pre-trained weights, so the library warns us that we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate an `IPUTrainer` class, we will need to define: 
* `IPUConfig`, which is a class that specifies attributes and configuration parameters to compile and put the model on the device.
* `IPUTrainingArguments`, which is a class that contains all the attributes to customize the training.
* A data collator.
* How to compute the metrics from the predictions.

We initialize `IPUConfig` with a config name or path, which we set earlier:

In [ ]:
ipu_config = IPUConfig.from_pretrained(
    ipu_config_name, executable_cache_dir=executable_cache_dir
)

`IPUTrainingArguments` requires a folder name, which will be used to save the checkpoints of the model. All other arguments are optional:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = IPUTrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=micro_batch_size,
    per_device_eval_batch_size=micro_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    n_ipu=n_ipu,
    num_train_epochs=3,
    weight_decay=0.01,
    dataloader_drop_last=True,
    logging_steps=10,
    push_to_hub=False,
)

Here we have set the evaluation to be done at the end of each epoch, tweak the learning rate, use the three batch-size-related arguments defined earlier in the notebook and customize the number of epochs for training, as well as the weight decay.

`push_to_hub` in `IPUTrainingArguments` is necessary if we want to push the model to the [🤗 Model hub](https://huggingface.co/models) regularly during training. You can remove them if you didn't follow the installation steps at the beginning of this notebook. If you want to save your model locally to a name that is different to the name of the repository it will be pushed to, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-ner"` or `"huggingface/bert-finetuned-ner"`).

Then we will need a data collator that does not do any additional preprocessing. This is because we have already done the padding during the earlier preprocessing stage.

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In order to compute the metrics from the predictions, we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [ ]:
metric = load_metric("seqeval")

This metric takes a list of labels for the predictions and references:

In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

So we will need to do a bit of post-processing on our predictions:
- Select the predicted index (with the maximum logit) for each token.
- Convert it to its string label.
- Ignore data where we have set a label of -100.

The following function does all this post-processing on the result of `IPUTrainer.evaluate` (which is a named tuple containing predictions and labels) before applying the metric:

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Now we are ready to instantiate our `IPUTrainer` class. We pass all of this along with our datasets to `IPUTrainer`:

In [ ]:
trainer = IPUTrainer(
    model,
    ipu_config,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

We can now fine-tune our model by calling the `train` method:

In [ ]:
trainer.train()

The `evaluate` method allows you to run the evaluation again on the evaluation dataset or on another dataset:

In [ ]:
trainer.evaluate()

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

You can now upload the result of the training to the 🤗 Hub:

In [ ]:
# trainer.push_to_hub()

You can also share this model and other users can load it with the identifier "your-username/the-name-you-picked" so for instance:

```python
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("sgugger/my-awesome-model")
```

## Next steps

Try out the other [IPU-powered Jupyter Notebooks](https://www.graphcore.ai/ipu-jupyter-notebooks) to see how how IPUs perform on other tasks.